# Fetching Weather Data and Uploading to AWS S3

This guide provides steps for fetching weather data using the OpenWeatherMap API and then uploading the data to an Amazon S3 bucket using Python.

## Step 1: Obtain an API Key from OpenWeatherMap

1. Register on the [OpenWeatherMap website](https://openweathermap.org/) and create an account.
2. Find and copy the API key from your account dashboard.

## Step 2: Write Python Function to Fetch Weather Data

In [1]:
import requests

def get_weather_data(city, api_key):
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {'q': city, 'appid': api_key}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to fetch weather data")

## Step 3: Set Up AWS Credentials for S3 Access
* Install Boto3 using pip install boto3.
* Configure AWS credentials (AWS Access Key ID and Secret Access Key).

In [2]:
!pip install boto3

## Step 4: Write Python Function to Upload Data to S3


In [13]:
import boto3

bucket_name = "lab-03-group-2" 
region_name = "us-east-1"  # You can choose a different region if needed

# Create a boto3 session
session = boto3.Session()

# Create an S3 client
s3_client = session.client("s3", region_name=region_name)

try:
  # Create the S3 bucket
  s3_client.create_bucket(Bucket=bucket_name)
  print(f"S3 bucket '{bucket_name}' created successfully in region '{region_name}'.")
except ClientError as e:
  if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
    print(f"Bucket '{bucket_name}' already exists in your account.")
  else:
    # Other errors
    print(f"Error creating bucket: {e}")

S3 bucket 'lab-03-group-2' created successfully in region 'us-east-1'.


In [16]:
import json

region = 'us-east-1'

def upload_to_s3(bucket_name, file_name, data):
  """Uploads data to an S3 bucket with error handling.

  Args:
      bucket_name (str): Name of the S3 bucket.
      file_name (str): Name of the file to be uploaded within the bucket.
      data (object): The data to be uploaded (can be any serializable object).

  Returns:
      object: The response object from the S3 upload operation if successful, otherwise None.

  Raises:
      Exception: A generic exception with a message indicating the error that occurred.
  """

  try:
    response = s3_client.put_object(Bucket=bucket_name, Key=file_name, Body=json.dumps(data))
    return response

  except Exception as error:
    # Catch any other unexpected exceptions
    error_message = f"An unexpected error occurred: {error}"
    raise Exception(error_message)

## Step 5: Combine the Functions in a Script


In [17]:
# Main execution
api_key = "3cde20cc977551b5a38ce38685da1d7a"  
city = "London"  
file_name = "weather_data.json"

try:
    weather_data = get_weather_data(city, api_key)
    upload_to_s3(bucket_name, file_name, weather_data)
    print("Data uploaded successfully to S3")
except Exception as e:
    print(f"An error occurred: {e}")

Data uploaded successfully to S3


In [18]:
# Example usage
api_key = "3cde20cc977551b5a38ce38685da1d7a" 
city = "London"
try:
    weather_data = get_weather_data(city, api_key)
    print(weather_data)
except Exception as e:
    print(f"An error occurred: {e}")

{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 283.77, 'feels_like': 283, 'temp_min': 282.36, 'temp_max': 284.87, 'pressure': 1008, 'humidity': 81}, 'visibility': 10000, 'wind': {'speed': 4.02, 'deg': 233, 'gust': 6.26}, 'clouds': {'all': 56}, 'dt': 1712531354, 'sys': {'type': 2, 'id': 2075535, 'country': 'GB', 'sunrise': 1712553549, 'sunset': 1712601896}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}


# Assignment: Groups to Convert JSON to CSV

Step 1. Take the JSON output and convert it to a Dataframe using pandas
Step 2. Now upload the CSV file to the 'lab-03' S3 bucket in the cloud with the following naming convention: <your group name>_weather_date_london_<datetimestamp>.csv


#### Step 1

In [22]:
def flatten_json_stack(json_obj, prefix=''):
  """
  Flatten a JSON object using a stack, returning the flattened dictionary.

  Args:
      json_obj (dict): The JSON object to flatten.
      prefix (str): Prefix to prepend to flattened keys.

  Returns:
      dict: Flattened dictionary.
  """
  flat_dict = {}  # Initialize an empty dictionary to store flattened key-value pairs
  stack = [(prefix, json_obj)]  # Stack to hold (prefix, value) tuples

  while stack:
    current_prefix, current_value = stack.pop()  # Unwind from the stack

    if isinstance(current_value, dict):
      for key, value in current_value.items():
        new_prefix = current_prefix + key + '_'
        stack.append((new_prefix, value))  # Push nested objects onto stack
    elif isinstance(current_value, list):
      for i, item in enumerate(current_value):
        new_prefix = current_prefix + f'{i}_'
        stack.append((new_prefix, item))  # Push list elements onto stack
    else:
      flat_dict[current_prefix] = current_value  

  return flat_dict 

In [23]:
import pandas as pd
dict_flattened = flatten_json_stack(weather_data)

weather_df = pd.DataFrame([dict_flattened])

weather_df

,cod_,name_,id_,timezone_,sys_sunset_,sys_sunrise_,sys_country_,sys_id_,sys_type_,dt_,...,main_temp_min_,main_feels_like_,main_temp_,base_,weather_0_icon_,weather_0_description_,weather_0_main_,weather_0_id_,coord_lat_,coord_lon_
0,200,London,2643743,3600,1712601896,1712553549,GB,2075535,2,1712531354,...,282.36,283,283.77,stations,04n,broken clouds,Clouds,803,51.5085,-0.1257


#### Step 2

In [24]:
from io import StringIO

def upload_dataframe_to_s3(df, bucket_name, key):
  """
  Uploads a Pandas dataframe to S3 as a CSV file.

  Args:
    df (pandas.DataFrame): The dataframe to upload.
    bucket_name (str): The name of the S3 bucket to upload to.
    key (str): The key (filename) of the object in S3.
  """
  # Create a StringIO object to act as a buffer
  csv_buffer = StringIO()
  
  # Save the dataframe to the buffer as a CSV file
  df.to_csv(csv_buffer,index = False)

  # Create an S3 client session
  s3_client = boto3.client('s3')

  # Upload the CSV content to S3
  s3_client.put_object(Body=csv_buffer.getvalue(), Bucket=bucket_name, Key=key)


In [25]:
key = "weather_date_london.csv"

upload_dataframe_to_s3(weather_df, bucket_name, key)

print("DataFrame uploaded successfully!")

DataFrame uploaded successfully!


In [26]:
def list_bucket_contents(bucket_name):
  """
  Prints the contents (filenames) of an S3 bucket.

  Args:
    bucket_name (str): The name of the S3 bucket to list.
  """
  # Create an S3 client session
  s3_client = boto3.client('s3')
  
  # Get the list of objects in the bucket
  try:
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    contents = response.get('Contents', [])
    
    # Check if bucket is empty
    if not contents:
      print(f"Bucket '{bucket_name}' is empty.")
      return
    
    # Print the filename (key) of each object
    for obj in contents:
      print(obj['Key'])
  except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchBucket':
      # Handle non-existent bucket error
      print(f"Bucket '{bucket_name}' does not exist.")
    else:
      # Handle other errors
      raise

In [27]:
list_bucket_contents(bucket_name)

weather_data.json
weather_date_london.csv
